In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']

dummy_theory_output = torch.tensor(np.median(dummy_data))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)])

theory_output = torch.tensor(np.median(data))
lower_to_median = torch.tensor(np.sort(data)[int(len(data)/2)-1])
upper_to_median = torch.tensor(np.sort(data)[int(len(data)/2)])

In [6]:
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        # since within 1%, we regard as same value
        count_less = torch.sum((X < 0.99*self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))
        
        # not support modulo yet
        less_cons = count_less<half_len+2*(len/2 - torch.floor(len/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<0.99*self.upper).double())==half_len
        bound = 2*count_less== 2*half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))

        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)

    
verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

dummy output:  tensor(15.8000, dtype=torch.float64)


/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# prover calculates settings, send to verifier
print("Theory_output: ", theory_output)
print("lower: ", lower_to_median)
print("upper: ", upper_to_median)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        # since within 1%, we regard as same value
        count_less = torch.sum((X < 0.99*self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))
        
        # not support modulo yet
        less_cons = count_less<half_len+2*(len/2 - torch.floor(len/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<0.99*self.upper).double())==half_len
        bound = 2*count_less == 2*half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)


 
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory_output:  tensor(49.5500, dtype=torch.float64)
lower:  tensor(49.3000, dtype=torch.float64)
upper:  tensor(49.8000, dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-25112,24986],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":12052,"total_const_size":1815,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.857371807098389 seconds
Theory output:  tensor(49.5500, dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.55078125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18341455175509539295, 12796101019039945164, 1607286914885633240, 1929881192315725821]]], 'proof': '25cc712d6d34992fbead930e1e840a614b218f926129ecb7dadee79bf121f7d82d1a8de3b0e152d216a55a3e7a5bc6f5d7ece6e6917c7c29cc18ffd92f1df8db227f121c52ef77da33ed879fbc571ef6f26a50487695bdfbad32a239228869261b857ec74a2479102488d68f846b42b7f21f72ca9e5ffffadba545925f85fc89110867600929d17ca8d7cb962a4b13617cd27ef90b6ceed6e5f9fc42c4b7362a19f746c6e0d9eae85c892a22669a84fcb4532dde9f55b7060e6ff659ff7e349612183b4f220cdbe93468409fcbce06d10bed6c50ea22478618d9e8d5f9f6615c2272df2ea708fe6ef50f4048e8544b9a8cb7c395457a6381aad3a24dfdba21bd121efafedfdb00cf89d02c732e9e66cc22d202be49dabf6ff6d9f4018619d57218034f2a80e9639f3fea01826ec551aa3b705994e907b366cdbd6de2a3a26c1514c7143013a3eb4531d82d21f1ced6af190d4dc6b54cb26ffbe96ecde578dd44

In [11]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

49.55078125